In [ ]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from scipy import stats

In [ ]:
# Import warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mengakses file dari drive
from google.colab import drive
drive.mount ('/content/drive')

In [ ]:
# Mengakses file dataset
file_path = '/content/drive/MyDrive/SQL/final_projects.csv'

olist = pd.read_csv(file_path)

# Preview dataset
olist.head()

In [ ]:
# Display maksimum kolom dan baris
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Ekstrak ukuran data
nrows, ncols = olist.shape

print(f"Data Input : ({nrows}, {ncols}) - (#jumlah observasi, #jumlah fitur)")

Dapat dilihat bahwa di dalam dataset terdapat 13.235 observasi, dengan 11 kolom

In [ ]:
# Cek informasi umum dari data
olist.info()

Dapat dilihat bahwa pada dataset, terdapat null value pada kolom category dan review_score. Dapat dilihat pula jika time_purchased masih berbentuk object, yang mana seharusnya berbentuk date-time

In [ ]:
# Cek data duplikat

olist.duplicated().any()

In [ ]:
# Menghilangkan data duplikat

olist = olist.drop_duplicates(keep='first')

olist.info()

Setelah dilakukan pengecekan terhadap data duplikat, data yang tersisa ada 7751 entries

In [ ]:
# Cek missing value

olist.isna().any().sum()

Terdapat dua kolom dengan missing value

In [ ]:
# Mencari kolom dengan missing value
col_missing = []

for col in olist.columns:
  if olist[col].isna().any() == True:
    col_missing.append(col)

col_missing

Kolom dengan missing value adalah kolom `category` dan `review_score`


In [ ]:
# Mendapatkan persentase jumlah missing value tiap kolom
nan_col = olist.isna().sum().sort_values (ascending=False) # Jumlah missing value per kolom

n_data = len (olist)

percent_nan_col = (nan_col/n_data) * 100 # Besar persentase
percent_nan_col

*Entry dengan missing value pada category dihapus karena tidak mungkin untuk mengimputasi menggunakan median, modus, atau mean sebab kategori produk bisa sangat spesifik; dan dikarenakan jumlahnya tidak terlalu banyak sehingga tidak akan terlalu mempengaruhi keseluruhan dataset*

In [ ]:
# Menghapus entry yang memiliki missing value pada 'category'

olist = olist.dropna(subset=['category'])

*Selanjutnya menginput missing value pada review score dengan median yang ada pada tiap product category sehingga tidak terlalu jauh dengan category-nya*

In [ ]:
# Mengganti missing review_score dengan median pada tiap product category


med_review = olist.groupby('category')['review_score'].median() # Mengkalkulasi median tiap kategori

med_review_dict = dict(med_review) # Menyimpan kalkulasi median tiap kategori

def impute_median(row): # Function untuk memasukan pada missing value
    if pd.isnull(row['review_score']):
        return med_review_dict[row['category']]
    else:
        return row['review_score']

olist['review_score'] = olist.apply(impute_median, axis=1) # Menjalankan function

olist.info() # Melihat info dataset setelah dilakukan manipulasi data


Setelah dilakukan pengecekan, masih ada missing value pada review score. Oleh karena itu dilakukan pengecekan pada data dengan missing score dan dictionary yang menyimpan nilai median.

In [ ]:
# List untuk menunjukan data review_score yang kosong
missing_review_score = olist[olist['review_score'].isnull()]

# Menunjukan hasil
print(missing_review_score)

In [ ]:
med_review_dict # Nilai median untuk setiap kategori

Setelah ditelusuri, ternyata terdapat typo pada `category` sehingga tidak terbaca datanya (home_comfort_2) dan terdapat kategori yang keseluruhannya tidak ada review score-nya (signaling_and_security dan cds_dvd_musical). Akan dilakukan input secara manual dengan value pada kategori yang mirip. (home_comfort_2 > home_confort , signaling_and_security > construction_tools_safety , cds_dvd_musical > dvd_bluray).

In [ ]:
# Membuat dict untuk nilai yang akan dimasukan
category_med = {'home_comfort_2': 4.0, 'signaling_and_security': 4.5, 'cds_dvds_musicals': 4.5, 'diapers_and_hygiene' : 5.0}

# Menginput nilai pada dataset
olist['review_score'] = olist.apply(lambda x: category_med[x['category']] if pd.isnull(x['review_score']) else x['review_score'], axis=1)


In [ ]:
# Check final dataset info
olist.info() # Sudah tidak terdapat missing value

Selanjutnya akan dilakukan data cleansing untuk inconsistent format

***Cleansing pada kolom `category`***

In [ ]:
# List entry unik kategori

olist['category'].unique()

In [ ]:
# Dictionary untuk revisi entry kategori

revisi_category = {
    'telephony' : 'telephone',
    'home_confort' : 'home_comfort',
    'costruction_tools_garden' : 'construction_tools_garden',
    'fashio_female_clothing' : 'fashion_female_clothing',
    'home_appliances_2' : 'home_appliances',
    'costruction_tools_tools' : 'construction_tools_tools',
    'home_comfort_2' : 'home_comfort'
}

# Melakukan revisi

olist['category'] = olist['category'].replace(revisi_category)

***Cleansing untuk `payment_type`***

In [ ]:
# List entry unik payment_type
olist['payment_type'].unique()

***Cleansing untuk `city`***

In [ ]:
# List entry unik city
olist['city'].unique()

***Cleansing untuk `state`***

In [ ]:
# List entry unik state
olist['state'].value_counts()

***Mengubah data type string menjadi datetime***

In [ ]:
# Mengubah menjadi datetime
olist['time_purchased'] = pd.to_datetime(olist['time_purchased'])

# Mengecek dataset
olist.info()

***Exploratory Data Analysis***

**Top Product Category**

In [ ]:
#Filter kategori yang paling banyak terjual
sorted_category = olist['category'].value_counts().sort_values(ascending=False) # Mengurutkan data berdasarkan jumlah item yang terjual
top_category = sorted_category[:20].sort_values(ascending=False) # Memfilter data 20 teratas
olist_top_category = olist[olist['category'].isin(top_category.index)] # Membuat dataframe untuk data 20 teratas

In [ ]:
#Filter kategori dengan penjualan paling besar
category_sales = olist.groupby('category')['price'].sum().sort_values(ascending=False) # Menghitung dan mengurutkan data berdasarkan besar penjualan
top_category_sales = category_sales[:20] # Memfilter data 20 teratas

olist_top_sales = olist[olist['category'].isin(top_category_sales.index)] # Membuat dataframe untuk data 20 teratas

In [ ]:
# Untuk mengatur jumlah subplots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

# Untuk membuat histogram kategori yang paling banyak terjual
sns.histplot(data=olist_top_category, x='category', bins=len(top_category), ax=ax[0])
ax[0].tick_params(labelrotation=90)
ax[0].set_title("Top Category Sold")

# Untuk membuat barplot kategori dengan penjualan paling besar
sns.barplot(data=olist_top_sales, x=top_category_sales.index, y=top_category_sales.values, ax=ax[1])
ax[1].tick_params(labelrotation=90)
ax[1].set_title("Top Sales by Category")

plt.show()

**Sales trend by year**

In [ ]:
# Membuat index datetime dan dataframe baru berdasarkan tahun

olist.set_index('time_purchased', inplace=True)

olist_2017 = olist.loc['2017']
olist_2018 = olist.loc['2018']

In [ ]:
# Mengatur jumlah plot 
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Plot time series untuk tahun 2017, dan 2018
for i, year in enumerate([olist_2017, olist_2018]):
    top_category_sales = category_sales[:7]
    year_top_category_sales = year[year['category'].isin(top_category_sales.index)]
    sns.lineplot(data=year_top_category_sales, x=year_top_category_sales.index.month, y='price', hue='category', ax=ax[i])
    ax[i].set_title('Top 7 Categories Sales in {}'.format(year.index.year[0]))
    ax[i].set_xlabel('Month')
    ax[i].set_ylabel('Total Sales')

plt.tight_layout()
plt.show()

**Correlation between top category and top city**

In [ ]:
# Filter data top 15 dari 'category' dan 'city' 
top_category = olist['category'].value_counts().nlargest(15).index
top_city = olist['city'].value_counts().nlargest(15).index

In [ ]:
# Membuat dataframe baru berdasarkan top category dan top city
olist_city_category = olist.loc[olist['category'].isin(top_category) & olist['city'].isin(top_city)]

In [ ]:
# Membuat pivot table
pivot_table = olist_city_category.pivot_table(index='category', columns='city', values='price', aggfunc='sum')

In [ ]:
# Mengatur ukuran plot
fig, ax = plt.subplots(figsize=(10,8))

# Membuat plot heatmap
sns.heatmap(pivot_table, cmap='YlGnBu', annot=True, fmt='.0f', ax=ax)
ax.set_title('Correlation between city and product category based on sales')

plt.show()

In [ ]:
# Mendowload dataset yang telah dibersihkan
olist.to_csv('/content/drive/MyDrive/SQL/final_projects_cleaned2.csv', index=False)

from google.colab import files
files.download('/content/drive/MyDrive/SQL/final_projects_cleaned2.csv')